In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def detect_encoding(file_path):
    # Try reading the file with different encodings
    encodings = ['utf-8', 'latin1', 'cp1252', 'windows-1252', 'ISO-8859-1']  # Add more encodings as necessary

    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return encoding
        
        except UnicodeDecodeError:
            continue  # Try next encoding if current one fails

    # If all encodings fail
    raise ValueError("Could not determine the encoding of the CSV file")

In [3]:
def handle_columns(df):
    
    try: 
        df.insert(0,"Kopfartikelnummer",np.nan)
        df.insert(1,"Kopfartikelname",np.nan)
        df.insert(2,"Kopfartikelbeschreibung",np.nan)
        df.insert(3,"Artikelname",np.nan)
        df.insert(4,"Artikelbeschreibung",np.nan)
        df.insert(5,"Positionsnummer",np.nan)
        df.rename(columns={"Bezeichnung":"Artikelnummer","Menge":"Anzahl"},inplace=True)
    except Exception as e:
        print(e)

In [4]:
def get_grid_type_and_colour(df_grouped):
    dummy = df_grouped[df_grouped["Artikelnummer"].str.startswith(("G.", "GF"))]
    
    typ = dummy.iloc[0]["Artikelnummer"].split(".")[0]

    if dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NB":
        colour = "NB"
    elif dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NG":
        colour = "NG"
    elif dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NW":
        colour = "NW"
    
    else:
        print("coudn't recognize colour")
    
    return typ,colour

In [6]:
# loop through csvs, insert /delete the right columns and insert kopfartikelnummer

for file in os.listdir():
    if file.endswith(".csv"):
        
        kopfartikelnummer = file.split(".")[0]
        encoding = detect_encoding(file)
        
        try:
            artikelliste_df = pd.read_csv(file, sep=";", encoding=encoding)
        except Exception as e:
            print(e)
        
        # get rid of empty Strings
        artikelliste_df = artikelliste_df.apply(lambda x: x.replace("", None))
        
        #query and group
        artikelliste_df.query("Pos.notnull()", inplace=True)
        artikelliste_df_grouped = artikelliste_df.groupby("Bezeichnung")["Menge"].sum().reset_index()
        
        handle_columns(artikelliste_df_grouped)
        
        artikelliste_df_grouped["Kopfartikelnummer"] = kopfartikelnummer

        artikelliste_df_grouped.to_csv(f"finished_{kopfartikelnummer}.csv", index=False, sep=";", encoding="ISO-8859-1")

In [7]:
# test SM4
if not artikelliste_df_grouped[artikelliste_df_grouped["Artikelnummer"].str.startswith("SM4")].empty:
    # test existing Grids and their colours 
    if artikelliste_df_grouped[artikelliste_df_grouped["Artikelnummer"].str.startswith(("G.", "GF"))].shape[0] > 1:
        print("The Cubes in this configuration do have different colours or are mixed with flame retardned and normal. The special case about SM4 cannot be solved now. Be aware.")
    else: 
        print("code will be executed")
        # normale Grids minus 2 
        artikelliste_df_grouped.loc[artikelliste_df_grouped["Artikelnummer"].str.startswith(("G.", "GF")), "Anzahl"] -= 2

        # spezielle Grid + 1 je nach farbe
        typ, colour = get_grid_type_and_colour(artikelliste_df_grouped)
        artikelnummer = typ + "." + colour + "_SM4"
        artikelliste_df_grouped.loc[len(artikelliste_df_grouped)]  = {"Kopfartikelnummer":kopfartikelnummer, "Artikelnummer":artikelnummer , "Anzahl":1}
        # special names in weclapp end with _SM4

code will be executed


In [8]:
artikelliste_df_grouped

,Kopfartikelnummer,Kopfartikelname,Kopfartikelbeschreibung,Artikelname,Artikelbeschreibung,Positionsnummer,Artikelnummer,Anzahl
0,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,AP,26.0
1,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,GF.NW,6.0
2,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,GRIDG7B,1.0
3,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,PB.MG,4.0
4,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,PBI.G,1.0
5,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,SM.MSB,1.0
6,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,SM4.MB00.0LH,1.0
7,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,SS.MB,10.0
8,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,SS.MG,1.0
9,spezialanfertigungen,NaN,NaN,NaN,NaN,NaN,GF.NW_SM4,1.0


In [ ]:
input("Press Enter to finish the script")